In [1]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import string

In [2]:
# Data Importing
df = pd.read_excel('new.xlsx')
df = df[df['Start_time'].notna()]
df = df.loc[df['transcript_needed']==1,:]
df.index = range(len(df))

In [3]:
df

,comp_name,description,source,title,url,fn_name,duration_in_script,TIME_in_script,DATE,rdq,...,MULTI_DUP,DIFF_PROG,PROGRAM,transcript_needed,Start_time,End_time,Time_in_EST_Other,achived url,archived_missing,Replay
0,UBS,"Sergio Ermotti, chief executive officer at UBS...",bb,"UBS' CEO on 2Q Earnings, Cost Cutting, Brexit",https://www.bloomberg.com/news/videos/2017-07-...,553.txt,663,01:58:56,2017-07-28,2017-07-28,...,NaN,NaN,Bloomberg Daybreak: Europe,1,02:35:15,02:37:50,EDT,https://archive.org/details/BLOOMBERG_20170728...,0,NaN
1,Agco Corp,"Martin Richenhagen, Agco Corp. chairman and ch...",bb,Agco CEO on Agricultural-Equipment Market and ...,https://www.bloomberg.com/news/videos/2017-08-...,567.txt,421,20:31:08,2017-07-31,2017-07-27,...,NaN,NaN,Bloomberg Markets,1,14:30:00,14:37:25,EDT,https://archive.org/details/BLOOMBERG_20170731...,0,NaN
2,HSBC USA,"Pat Burke, president and chief executive offic...",bb,HSBC USA CEO Sees Nearly Unlimited Growth in C...,https://www.bloomberg.com/news/videos/2017-08-...,580.txt,215,13:14:46,2017-08-02,2017-07-31,...,NaN,NaN,Bloomberg Daybreak: Americas,1,07:52:30,07:56:18,EDT,https://archive.org/details/BLOOMBERG_20170802...,0,NaN
3,Intesa Sanpaolo,"Carlo Messina, chief executive officer at Inte...",bb,"Intesa CEO Messina on Veneto Banks, Organic Gr...",https://www.bloomberg.com/news/videos/2017-08-...,574.txt,831,05:28:16,2017-08-02,2017-08-01,...,1.0,NaN,Bloomberg Surveillance,1,04:32:30,04:47:00,EDT,https://archive.org/details/BLOOMBERG_20170802...,0,NaN
4,Square Inc,"Square Inc.’s lending business is booming, but...",bb,Dorsey's Fight Against Economic Disparity,https://www.bloomberg.com/news/videos/2017-08-...,584.txt,286,23:18:16,2017-08-02,2017-08-02,...,NaN,NaN,Bloomberg Technology,1,12:14:30,12:19:20,EDT,https://archive.org/details/BLOOMBERG_20170803...,0,NaN
5,Aviva Plc,"Mark Wilson, chief executive officer at Aviva,...",bb,Aviva's CEO Says Very Happy With Current Compa...,https://www.bloomberg.com/news/videos/2017-08-...,585.txt,362,03:36:16,2017-08-03,2017-08-03,...,NaN,NaN,Bloomberg Daybreak: Europe,1,02:02:15,02:08:30,EDT,https://archive.org/details/BLOOMBERG_20170803...,0,NaN
6,Axa SA,"Thomas Buberl, chief executive officer at AXA,...",bb,AXA's CEO Says on Track With Our 2020 Ambition,https://www.bloomberg.com/news/videos/2017-08-...,586.txt,374,03:43:06,2017-08-03,2017-08-03,...,NaN,NaN,Bloomberg Daybreak: Europe,1,02:08:48,02:15:15,EDT,https://archive.org/details/BLOOMBERG_20170803...,0,NaN
7,Randgold Resources,"Mark Bristow, chief executive officer at Randg...",bb,"Randgold's CEO on Earnings, Organic Growth, Go...",https://www.bloomberg.com/news/videos/2017-08-...,587.txt,316,03:51:21,2017-08-03,2017-08-03,...,NaN,NaN,Bloomberg Daybreak: Europe,1,02:20:50,02:26:23,EDT,https://archive.org/details/BLOOMBERG_20170803...,0,NaN
8,DBS Group,DBS Group CEO Piyush Gupta discusses the compa...,bb,DBS CEO Gupta Says Digitization Is Driving Rev...,https://www.bloomberg.com/news/videos/2017-08-...,604.txt,228,19:59:46,2017-08-06,2017-08-03,...,NaN,NaN,Bloomberg Daybreak: Asia,1,20:22:30,20:26:20,EDT,https://archive.org/details/BLOOMBERG_20170806...,0,NaN
9,DISNEY (WALT) CO,Disney Chairman and CEO Robert Iger discusses ...,bb,Disney’s Iger Sees Dramatic Shift in Media Con...,https://www.bloomberg.com/news/videos/2017-08-...,5405.txt,155,18:25:51,2017-08-08,2017-08-08,...,1.0,NaN,Bloomberg Technology,1,17:46:20,18:00:05,EDT,https://archive.org/details/BLOOMBERG_20170808...,0,NaN


In [142]:
# Searching Rows
df.loc[df['achived url']=='https://archive.org/details/CNBC_20171101_220000_Mad_Money/start/1020/end/1080', :]

,comp_name,description,source,title,fn_name,duration,TIME_in_script,DATE,MULTI_ADD,MULTI_DUP,...,transcript_needed,Start_time,End_time,Time_in_EST_Other,achived url,archived_missing,Short?,download,duration_in_script,url
181,Allergan plc,Jim Cramer and Allergan CEO Brent Saunders dis...,cnbc,Allergan CEO on patent deal: 'It wasn't desper...,1982.txt,7.3,18:43:44,2017-11-01,NaN,1.0,...,1,18:17:12,18:23:00,EDT,https://archive.org/details/CNBC_20171101_2200...,0,NaN,NaN,438,/video/2017/11/01/allergan-ceo-on-patent-deal-...


In [5]:
def gettxt(id):
    x = df.loc[id, 'Start_time']
    x = str(x)
    oph = int(x[0:2])
    y = df.loc[id, 'End_time']
    y = str(y)
    edh = int(y[0:2])
    
    if oph>=13:
        oph-=12
        x=str(oph)+x[2:5]+' pm'
    elif oph==12:
        x=x[0:5]+' pm'
    elif oph<10:
        x=x[1:5]+' am'
    else:
        x=x[0:5]+' am'

    if edh>=13:
        edh-=12
        y=str(edh)+y[2:5]+' pm'
    elif edh==12:
        y=y[0:5]+' pm'
    elif edh<10:
        y=y[1:5]+' am'
    else:
        y=y[0:5]+' am'
        
    source = df.loc[id, 'source']
    name = df.loc[id, 'fn_name']
    url = df.loc[id, 'achived url']
    
    header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36 Edg/98.0.1108.43'}
    request = urllib.request.Request(url, headers=header)
    response = urllib.request.urlopen(request)
    html = response.read().decode('utf-8')
    bs = BeautifulSoup(html, 'html.parser')
    list = []
    for i in bs.div.strings:
        list.append(i.strip())
    while '' in list:
        list.remove('')
    with open('D:/Stevens/RA/Work 2 caption/txt_1/{}'.format(source+'_'+name), 'a+', encoding='utf-8') as f:
        f.write(url+'\n')
        for i in list[list.index(x):list.index(y)+2]:
            f.write(i+'\n')  

In [114]:
# Original Coding

# x = df.loc[124, 'Start_time']
# x = str(x)
# oph = int(x[0:2])
# y = df.loc[124, 'End_time']
# y = str(y)
# edh = int(y[0:2])

# if oph>=13:
#     oph-=12
#     x=str(oph)+x[2:5]+' pm'
# elif oph==12:
#     x=x[0:5]+' pm'
# elif oph<10:
#     x=x[1:5]+' am'
# else:
#     x=x[0:5]+' am'

# if edh>=13:
#     edh-=12
#     y=str(edh)+y[2:5]+' pm'
# elif edh==12:
#     y=y[0:5]+' pm'
# elif edh<10:
#     y=y[1:5]+' am'
# else:
#     y=y[0:5]+' am'

# source = df.loc[i, 'source']
# name = df.loc[i, 'fn_name']
# url = df.loc[i, 'achived url']
# header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36 Edg/98.0.1108.43'}
# request = urllib.request.Request(url, headers=header)
# response = urllib.request.urlopen(request)
# html = response.read().decode('utf-8')
# bs = BeautifulSoup(html, 'html.parser')
# list = []
# for i in bs.div.strings:
#     list.append(i.strip())
# while '' in list:
#     list.remove('')
# for i in list[list.index(x):list.index(y)+2]:
#     f=open('D:/Stevens/RA/Work 2 caption/txt/{}'.format(source+'_'+name), 'a+', encoding='utf-8')
#     f.write(i+'\n')
#     f.close()

In [7]:
# Run!
for i in range(43):
    gettxt(i)

In [135]:
# Test
url = 'https://archive.org/details/CNBC_20170518_220000_Mad_Money/start/960/end/1020'
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36 Edg/98.0.1108.43'}
request = urllib.request.Request(url, headers=header)
response = urllib.request.urlopen(request)
html = response.read().decode('utf-8')
bs = BeautifulSoup(html, 'html.parser')
list = []
for i in bs.div.strings:
    list.append(i.strip())
while '' in list:
    list.remove('')
list

['Due to a planned power outage on Friday, 1/14, between 8am-1pm PST, some services may be impacted.',
 'Internet Archive logo',
 'A line drawing of the Internet Archive headquarters building façade.',
 'Search icon',
 'An illustration of a magnifying glass.',
 'Search icon',
 'An illustration of a magnifying glass.',
 'Upload icon',
 'An illustration of a horizontal line over an up pointing arrow.',
 'Upload',
 'User icon',
 "An illustration of a person's head and chest.",
 'Sign up',
 '|',
 'Log in',
 'Web icon',
 'An illustration of a computer application window',
 'Wayback Machine',
 'Texts icon',
 'An illustration of an open book.',
 'Books',
 'Video icon',
 'An illustration of two cells of a film strip.',
 'Video',
 'Audio icon',
 'An illustration of an audio speaker.',
 'Audio',
 'Software icon',
 'An illustration of a 3.5" floppy disk.',
 'Software',
 'Images icon',
 'An illustration of two photographs.',
 'Images',
 'Donate icon',
 'An illustration of a heart shape',
 'Donate'